In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 11 11:02:03 2021

@author: Christian
"""

from mip import Model, xsum, maximize, BINARY

from Co2_price import df_data
from electricity_demand import electricity_demand
from Electricity_Price0 import Electricity_price_pivot0
from Electricity_Price1 import Electricity_price_pivot1
from gas_price import gas_p
from heat_demand import heat_demand_pivot
from heat_price import heat_p


elec_demand = electricity_demand
elec_price_wholesale = Electricity_price_pivot0
elec_price_household_32_05 = Electricity_price_pivot1
co2_price = df_data
gas_price = gas_p
heat_demand_1 = heat_demand_pivot
heat_price_1 = heat_p


el_price = elec_price_wholesale # Electricity price
del_t = .25 # duration of time step
el_demand = electricity_demand # Electicity demand


gas_p = gas_price # Gas price for different plants
em_fc = 0.2 # emission factor
co2_p = co2_price # CO2 Price
capacity_el = [5, 6, 7] # maximum capacity of electricity for the power plants
capacity_ht = [5, 6, 7] # maximum capacity of heat for the power plants
heat_demand = heat_demand_1 # heat demand
heat_price = heat_price_1 # heat price
heat_ratio = 1.21 # heat ratio


eff_plants = [0.33, 0.4, 0.45] # the efficiency of the power plants


T = range(len(el_price))
I = range(len(gas_p))

m = Model("Maximizing profit", sense=maximize)
 

#variable
y_t = [[m.add_var(lb = 0) for i in I] for t in T] # Electricity generation
el_sold = [m.add_var(lb = 0) for t in T] # electricity sold 
x_t = [[m.add_var(lb = 0) for i in I] for t in T] # Fuel consumption
z_t = [[m.add_var(lb = 0) for i in I] for t in T] # Heat generation
# objective function

# Revenue = Electricity price * demand/generation
# Cost = Fuel price = gas price + emission_factor * co2 price

# Max Proft = Revenue - Cost



In [2]:
m.objective = xsum((el_price[t] * del_t * el_sold[t]) - ( x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t] * heat_price[t]) for t in T)

# constraints

for t in T:
    m += xsum(y_t[t][i] for i in I) == el_demand[t]
    
    for i in I:
        m += y_t[t][i] <= capacity_el[i] # electricity generation <= maximum capacity of electricity of the plant

        m += z_t[t][i] <= capacity_ht[i] # heat generation <= maximum capacity of heat of the plant 
        
        m += x_t[t][i] == y_t[t][i]/eff_plants[i] # fuel consumption = Electricity generation / efficiency of the plants 

        m += heat_demand[t] <= z_t[t][i] # heat demand >= Heat generation  

        m += y_t[t][i] == heat_ratio * z_t[t][i] # electricity generation >= heat ratio * heat generation

        m += y_t[t][i] == el_sold[t] + el_demand[t]



status = m.optimize()
obj= m.objective_value

41.709999999999994

In [3]:
status

<OptimizationStatus.OPTIMAL: 0>